In [1]:
from datetime import datetime, timezone, timedelta
import scrapbook as sb

import msticpy as mp
mp.init_notebook()

In [4]:
# papermill params
ip_address: str = ""
end = datetime.now(timezone.utc)
start = end - timedelta(days=2)

## IP Address Basic data

In [35]:
ip_basic_df = (
    IpAddress.ip_type(ip_address)
    .mp_pivot.run(IpAddress.whois, column="ip", join="left")
)
display(ip_basic_df.T)


,0
ip,85.214.149.236
result,Public
src_row_index,0
asn,6724
query,85.214.149.236
asn_cidr,85.214.0.0/15
asn_country_code,DE
asn_registry,ripencc
asn_date,2005-02-24
asn_description,"STRATO STRATO AG, DE"


## IP Location

In [36]:
display(IpAddress.geoloc(ip_address).T)
ip_basic_df.mp_plot.folium_map(ip_column="ip", tooltip_columns=["ip", "CountryCode"])

,0
CountryCode,DE
CountryName,Germany
Longitude,9.491
Latitude,51.2993
TimeGenerated,2022-08-12 23:00:05.521005
Type,geolocation
IpAddress,85.214.149.236


## Threat Intelligence reports

In [34]:

ti_df = IpAddress.tilookup_ip(ip_address)
display(ti_df)

Observables processed: 100%|██████████| 7/7 [00:01<00:00,  5.62obs/s]


,Ioc,IocType,SanitizedValue,QuerySubtype,Provider,Result,Severity,Details,RawResult,Reference,Status
0,85.214.149.236,ipv4,85.214.149.236,None,GreyNoise,False,information,Not found.,<Response [404 Not Found]>,https://api.greynoise.io/v3/community/85.214.149.236,404
0,85.214.149.236,ipv4,85.214.149.236,None,OTX,True,high,"{'pulse_count': 44, 'names': ['Comprehensive Threat Intelligence: TeamTNT targeting AWS, Alibaba...","{'whois': 'http://whois.domaintools.com/85.214.149.236', 'reputation': 0, 'indicator': '85.214.1...",https://otx.alienvault.com/api/v1/indicators/IPv4/85.214.149.236/general,0
0,85.214.149.236,ipv4,85.214.149.236,None,RiskIQ,True,high,"{'summary': {'resolutions': 5, 'certificates': 6, 'malware_hashes': 0, 'projects': 0, 'articles'...","{'summary': {'resolutions': 5, 'certificates': 6, 'malware_hashes': 0, 'projects': 0, 'articles'...",https://community.riskiq.com,0
0,85.214.149.236,ipv4,85.214.149.236,None,Tor,True,information,Not found.,None,https://check.torproject.org/exit-addresses,0
0,85.214.149.236,ipv4,85.214.149.236,None,VirusTotal,True,high,"{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'positives': 32, 'detected_urls': [...","{'asn': 6724, 'undetected_urls': [['http://h2381205.stratoserver.net/', '011bcc2795245bb9fac15c5...",https://www.virustotal.com/vtapi/v2/ip-address/report,0
0,85.214.149.236,ipv4,85.214.149.236,None,XForce,True,information,"{'score': 1, 'cats': {}, 'categoryDescriptions': {}, 'reason': 'Regional Internet Registry', 're...","{'ip': '85.214.149.236', 'history': [{'created': '2012-03-22T07:26:00.000Z', 'reason': 'Regional...",https://api.xforce.ibmcloud.com/ipr/85.214.149.236,0
0,85.214.149.236,ipv4,NaN,None,AzSTI,False,0,Not found,NaN,ThreatIntelligenceIndicator | where TimeGenerated >= datetime(2022-08-11T21:27:57.291732Z) | w...,4


In [33]:
ti_high_sev = ti_df[ti_df["Severity"] == "high"]
if not ti_high_sev.empty:
    sb.glue("Findings", True)
    sb.glue(
        "TIProviders",
        str(ti_high_sev[["Provider", "Severity", "Reference"]].set_index("Provider"))
    )
    sb.glue("ip_basic", ip_basic_df.to_csv())
    sb.glue("ip_ti", ti_high_sev.to_csv())
